In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Import the Libraries**

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

In [3]:
dir1 = ('../input/brain-tumor-mri-dataset/Training')
category = []

for file in os.listdir(dir1):
    category.append(file)
    
category

In [4]:
for i in category:
    file_path = os.path.join(dir1,i)
    
    for file in os.listdir(file_path):
        img_path = os.path.join(file_path,file)
        image = cv2.imread(img_path)
    

In [5]:
plt.imshow(image)
print(image.shape)

**Lets see how image for each of the categories looks like**

In [6]:
plt.figure(figsize=(15,8))

img_path = ['/pituitary/Tr-piTr_0001.jpg','/notumor/Tr-noTr_0001.jpg','/meningioma/Tr-meTr_0001.jpg','/glioma/Tr-glTr_0001.jpg']

for i in range(4):
    ax = plt.subplot(2, 2, i + 1)
    img = cv2.imread(dir1 + img_path[i])
    img = cv2.resize(img, (224, 224))
    plt.imshow(img)
    plt.title(category[i])

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

**We will us the ImageDataGenerator module for Online data augmentation**

In [8]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    featurewise_center=False,
                                    samplewise_center=False,
                                    featurewise_std_normalization=False,
                                    samplewise_std_normalization=False,
                                    zca_whitening=False,
                                    rotation_range=0.2,
                                    zoom_range = 0.3,
                                    width_shift_range=0.2,
                                    height_shift_range=0,
                                    horizontal_flip=True,
                                    vertical_flip=False)


test_datagen = ImageDataGenerator(rescale=1./255,
                                    featurewise_center=False,
                                    samplewise_center=False,
                                    featurewise_std_normalization=False,
                                    samplewise_std_normalization=False,
                                    zca_whitening=False,
                                    rotation_range=0,
                                    zoom_range = 0,
                                    width_shift_range=0,
                                    height_shift_range=0,
                                    horizontal_flip=True,
                                    vertical_flip=False)

In [9]:
train = train_datagen.flow_from_directory('../input/brain-tumor-mri-dataset/Training',target_size=(200,200),class_mode='categorical',batch_size=32,color_mode='grayscale')
test=test_datagen.flow_from_directory('../input/brain-tumor-mri-dataset/Testing',target_size=(200,200),class_mode='categorical',batch_size=32,color_mode='grayscale')

**So now lets import everything that we need to create our CNN model**

In [10]:
from tensorflow.keras import models,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.activations import relu,softmax
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping


In [11]:
model = Sequential()

# Convolutional layer 1
model.add(Conv2D(64,(7,7), input_shape=(200, 200, 1), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

#Convolutional layer 2
model.add(Conv2D(128,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolutional layer 3
model.add(Conv2D(128,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolutional layer 4
model.add(Conv2D(256,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

 # Convolutional layer 5
model.add(Conv2D(256,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

# Convolutional layer 6
model.add(Conv2D(512,(7,7), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Full connect layers

model.add(Dense(units= 1024, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(units=4, activation='softmax'))



model.compile(optimizer='adam', loss='categorical_crossentropy',
                   metrics= ['categorical_accuracy'])

model.summary()


In [12]:
model_es = EarlyStopping(monitor = 'loss', min_delta = 1e-11, patience = 12, verbose = 1)

history = model.fit(train, epochs=100, validation_data=test,callbacks=model_es)

In [17]:
model.save('Braintumormodel.h5')

In [18]:
path='Weights_folder/Weights'
model.save_weights(path)

In [24]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,8))
plt.plot(history.history["loss"],c = "blue")
plt.plot(history.history["val_loss"],c = "green")
plt.legend(["train", "test"])
plt.title('Loss')

In [25]:
import matplotlib.pyplot as plt
plt.figure(figsize=(15,8))
plt.plot(history.history["categorical_accuracy"],c = "blue")
plt.plot(history.history["val_categorical_accuracy"],c = "green")
plt.legend(["train", "test"])
plt.title('Accuracy')